In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D, MaxPool2D, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model, Input
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.applications.inception_v3 import InceptionV3

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/cyclone_dataset/insat_3d_ds - Sheet.csv')
df.head()

In [ ]:
paths = glob.glob("/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL/*.jpg")
i=1 
for path in paths[:30]:
  fig = plt.figure(figsize=(25, 35))
  ax = fig.add_subplot(6, 5, i)
  img = f"image {i}"
  plt.imshow(load_img(path))
  if i<10:
    plt.title(f"10{i}.jpg")
  else:
    plt.title(f"1{i}.jpg")
    
  plt.xlabel(img.upper())
  i+=1
plt.savefig("raw_img")


In [ ]:
hm_path = glob.glob("/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/*.jpg")
i=1
for path in hm_path[:24]:
  fig = plt.figure(figsize=(20,30))
  ax = fig.add_subplot(6,4,i)
  plt.imshow(load_img(path))
  if i<10:
    ax.set_title(f"10{i}")

  else:
    ax.set_title(f"1{i}")

  ax.set_xlabel(f"IR_IMAGE - {i}")
  i+=1
plt.savefig("cyclone_infrared_img.svg")

In [ ]:
ref_path = glob.glob("/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_for_reference_ds/CYCLONE_DATASET/*.jpeg")
i=1
fig = plt.figure(figsize=(20,30))

for path in ref_path[:24]:
  ax = fig.add_subplot(6,4,i)
  plt.imshow(load_img(path))
  ax.set_title(f"{100+i}")


  ax.set_xlabel(f"IR_IMAGE - {i}")
  i+=1
  plt.axis("off") 

plt.savefig("each_image_analysis.svg")

In [ ]:
def dataset_preparation(path):
    x_train=[]
    y_train=[]
    all_images_path=glob.glob(path+'/*.jpg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
    return np.array(x_train)

In [ ]:
path = "/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL"
test_path = "/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED"

train_data = dataset_preparation(path)

test_data = dataset_preparation(test_path)

print(f"Shape of train data is {train_data.shape}.")
print(f"Size of train data is {len(train_data)}.")
print(f"Shape of test data is {test_data.shape}.")
print(f"Size of test data is {len(test_data)}.")

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
training_data = train_datagen.flow_from_dataframe(dataframe=df,directory='/content/drive/MyDrive/Datasets/cyclone_dataset/insat3d_raw_cyclone_ds/CYCLONE_DATASET_FINAL/',
                                                  subset="training",
                                               x_col="img_name",y_col="label",target_size=(512, 512),batch_size=16,class_mode='raw')

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(units=16,activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(units=8,activation='relu'))
model.add(Dropout(0.33))
model.add(Dense(1,activation='relu'))

model.summary()

In [ ]:
model.compile(optimizer= Adam(learning_rate=0.01),loss='mae',metrics=['RootMeanSquaredError'])


In [ ]:
history = model.fit_generator(
       training_data,
       epochs=10,
       )

In [ ]:
pre_trained_model = InceptionV3(weights="imagenet", include_top=False, 
                                input_tensor=Input(shape=(512, 512, 3)))
pre_trained_model.summary()

pre_trained_model.trainable = False

flatten = pre_trained_model.output
flatten = Flatten()(flatten)

bboxHead = Dense(64, activation="relu")(flatten)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(1, activation="linear")(bboxHead)

model = Model(inputs=pre_trained_model.input, outputs=bboxHead)

In [ ]:
model.compile(optimizer = Adam(1e-3), 
             loss = 'mae', 
             metrics = [RootMeanSquaredError()])

call_back = ModelCheckpoint('Model.h5',monitor='loss',verbose=1,save_best_only=True)

# Compiling the model with optimizer and loss function

model.fit_generator(training_data,epochs=100,callbacks=[call_back])


In [ ]:
# Loading the model

model = models.load_model('Model.h5')

model.evaluate(training_data)

In [ ]:
predictions = pd.DataFrame(model.predict(training_data,verbose=1))
predictions

In [ ]:
# Changing the name of the column
predictions.columns = ['Intensity']
predictions.head()

In [ ]:
# Saving the predictions into csv
predictions.to_csv('cyclone_intensity')